# Obtenção e tratamento de dados


## Obtenção e carregamento de dados


In [ ]:
#importação de bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from tabulate import tabulate
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support

In [ ]:
#Carregamento de dados limpos da pasta ../data/
df = pd.read_csv('../Data/dataset.csv')

## Informações sobre o dataset


In [ ]:
#Exibição dos dados
df.head()

In [ ]:
#Exibição de informações sobre os dados
df.info()

In [ ]:
#Exibição de estatísticas descritivas
df.describe()

## Ajuste de tipos


In [ ]:
#Matricula
df['dt_matricula'] = pd.to_datetime(df['dt_matricula'])
#Saida
df['dt_saida'] = pd.to_datetime(df['dt_saida'])
#Primeiro login
df['primeiro_login'] = pd.to_datetime(df['primeiro_login'])
#Ultimo login
df['ultimo_login'] = pd.to_datetime(df['ultimo_login'])

#Idade
df['idade'] = df['idade'].astype('int64')
#Total de logins
df['total_logins'] = df['total_logins'].astype('int64')
#Dias desde o primeiro login
df['dias_desde_primeiro_login'] = df['dias_desde_primeiro_login'].astype('int64')
# Quantidade Mestrado
df['qtd_professores_mestrado'] = df['qtd_professores_mestrado'].astype('int64')
# Quantidade Doutorado
df['qtd_professores_doutorado'] = df['qtd_professores_doutorado'].astype('int64')
# Quantidade Especialização
df['qtd_professores_especialista'] = df['qtd_professores_especialista'].astype('int64')
# Quantidade Graduado
df['qtd_professores_graduado'] = df['qtd_professores_graduado'].astype('int64')
# Quantidade de professores Masculino
df['qtd_professores_M'] = df['qtd_professores_M'].astype('int64')
# Quantidade de professores Feminino
df['qtd_professores_F'] = df['qtd_professores_F'].astype('int64')
# Menor idade de um professor
df['idade_min_professores'] = df['idade_min_professores'].astype('int64')
# Maior idade de um professor
df['idade_max_professores'] = df['idade_max_professores'].astype('int64')

#Data de registro para string
df['dt_registro'] = df['dt_registro'].astype(object)
#Data de inauguração do curso
df['dt_inauguracao_curso'] = df['dt_inauguracao_curso'].astype(object)

#Exibição de informações sobre os dados
df.info()
#Exibição de estatísticas descritivas
df.head()


## Tratamento de valores ausentes


In [ ]:
#preenche com o texto "Abandono" os valores nulos da coluna 'categoria_motivo_cancelamento'
df['categoria_motivo_cancelamento'] = df['categoria_motivo_cancelamento'].fillna('Abandono').where(df['evasao'] == 1, 'NA')

#Exibe os valores unicos da coluna 'categoria_motivo_cancelamento'
df['categoria_motivo_cancelamento'].unique()

Exibição dos dados


In [ ]:
#Exibe as informações sobre os dados
df.info()
#Exibe as estatísticas descritivas
df.head()

# Estatísticas (Regressão logística) - Construção do modelo

## Amostra dos dados

In [ ]:
df.head()

## Codificação de variáveis categóricas e numéricas

In [ ]:
# Exclua as colunas que não serão usadas
df = df.drop(['cd_aluno','nome', 'curso','cidade','titulacao', 'estado', 'semestre_atual','dias_matriculado_semestre','total_logins','qtd_aprovacoes','valor_mensalidade_bruto', 'percentual_desconto','dias_desde_primeiro_login', 'dt_registro', 'dt_inauguracao_curso','dt_matricula', 'dt_saida', 'primeiro_login', 'ultimo_login', 'idade_min_professores', 'idade_max_professores'], axis=1)

In [ ]:
# Defina as colunas numéricas e categóricas
num_cols = ['idade', 'valor_mensalidade_liquido', 'qtd_mensalidade_paga_atraso', 'qtd_semestres_curso', 'qtd_reprovacoes', 'coeficiente_rendimento',  'media_acessos_por_dia', 'semestres_existencia_curso', 'qtd_professores', 'qtd_professores_mestrado', 'qtd_professores_doutorado', 'qtd_professores_especialista', 'qtd_professores_graduado', 'qtd_professores_M', 'qtd_professores_F', 'idade_media_professores']
cat_cols = ['genero', 'ds_ensino_medio', 'ds_ingresso', 'cd_curso', 'situacao', 'categoria_motivo_cancelamento']

## Definição do Pré-processador

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)])

## Definição do modelo

In [ ]:
model = LogisticRegression(max_iter=1000, penalty='l2', C=0.1)

## Combinação do pré-processador e do modelo em um pipeline

In [ ]:
clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

## Definição de variáveis independentes e dependentes


In [ ]:
X = df.drop(['evasao'], axis=1)
y = df['evasao']

## Divisão dos dados em treino e teste


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dividimos o conjunto de treinamento novamente para criar o conjunto de validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

- X_train e Y_train => Treinamento 
- X_val e Y_val => Validação
- X_test e Y_test => Teste

## Treinando e Validando o modelo

### Treinamento do modelo com os dados de treino

In [ ]:
clf.fit(X_train, y_train)

### Validação do modelo

In [ ]:
val_score = clf.score(X_val, y_val)
print(f'Acurácia da validação: {val_score}')

### Treinamento final do modelo com os dados de treino e validação juntos

In [ ]:
clf.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

### Teste do modelo

In [ ]:
test_score = clf.score(X_test, y_test)
print(f'Acurácia do teste: {test_score}')

# Estatísticas (Regressão logística) - Previsões e avaliação

## Fazendo previsões

In [ ]:
y_pred = clf.predict(X_test)

## Calculando a acurácia

#### Acurácia do modelo

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracia da previsão: {:.2f}'.format(accuracy))

#### Validação cruzada

In [ ]:
# Definindo o número de folds para a validação cruzada
cv = 5

# Realizando a validação cruzada
scores = cross_val_score(clf, X, y, cv=cv)

print(f'Acurácia da validação cruzada: {scores.mean()} +/- {scores.std()}')

## Matriz de confusão

In [ ]:
# Calculando a matriz de confusão
conf_mat = confusion_matrix(y_test, y_pred)

# Criando um gráfico
fig, ax = plt.subplots( figsize=(10, 6))

# Criando um mapa de calor
cax = ax.matshow(conf_mat, cmap=plt.cm.viridis, alpha=0.8)

# Adicionando os valores das células
for i, j in itertools.product(range(conf_mat.shape[0]), range(conf_mat.shape[1])):
    plt.text(j, i, f'{conf_mat[i, j]}', horizontalalignment='center', color='black', fontsize=14)

# Adicionando rótulos aos eixos
plt.xlabel('Valores Previstos')
plt.ylabel('Valores Verdadeiros')
plt.title('Matriz de Confusão')
plt.xticks(np.arange(2), ['Não evadidos', 'Evadidos'])
plt.yticks(np.arange(2), ['Não evadidos', 'Evadidos'])
plt.show()

## Relatório de classificação

In [ ]:
y_test_labels = y_test.map({0: 'Não Evadido', 1: 'Evadido'})
y_pred_labels = pd.Series(y_pred).map({0: 'Não Evadido', 1: 'Evadido'})

# Cria o relatório de classificação
report = classification_report(y_test_labels, y_pred_labels, output_dict=True)

# Criando um DataFrame a partir do relatório de classificação
report_df = pd.DataFrame(report).transpose()

# Exibindo as métricas de precisão, recall, f-score e suporte
report_df.head()